In [ ]:
import pandas as pd
# Step 2 'Credits, Genres, and Keywords Based Recommender' necessary libraries
from ast import literal_eval
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
metadata=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Content-Based-Recommender/movies/movies_small_metadata.csv')
metadata.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000.0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,No Tagline,Toy Story,False,7.7,5415.0
1,False,No Collection,65000000.0,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",No Homepage,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0.0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",No Homepage,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,No Collection,16000000.0,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",No Homepage,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0.0,"[{'id': 35, 'name': 'Comedy'}]",No Homepage,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


### Credits, Genres, and Keywords Based Recommender

#### Load & merge data

In [ ]:
credits = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Content-Based-Recommender/movies/credits.csv')
keywords = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Content-Based-Recommender/movies/keywords.csv')

In [ ]:
credits.info()
keywords.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45476 entries, 0 to 45475
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   cast    45476 non-null  object
 1   crew    45476 non-null  object
 2   id      45476 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 1.0+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46419 entries, 0 to 46418
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        46419 non-null  int64 
 1   keywords  46419 non-null  object
dtypes: int64(1), object(1)
memory usage: 725.4+ KB


In [ ]:
# Convert IDs to int. Required for merging
keywords['id'] = keywords['id'].astype('int')
credits['id'] = credits['id'].astype('int')
metadata['id'] = metadata['id'].astype('int')

# Merge keywords and credits into the main metadata dataframe
metadata = metadata.merge(credits, on='id')
metadata = metadata.merge(keywords, on='id')

In [ ]:
# Display the first few rows to verify the merge
metadata.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,spoken_languages,status,tagline,title,video,vote_average,vote_count,cast,crew,keywords
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000.0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,No Tagline,Toy Story,False,7.7,5415.0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,False,No Collection,65000000.0,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",No Homepage,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...","[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0.0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",No Homepage,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...","[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,False,No Collection,16000000.0,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",No Homepage,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...","[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0.0,"[{'id': 35, 'name': 'Comedy'}]",No Homepage,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...","[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."


#### Feature Extraction

From our new features, cast, crew, and keywords, we need to extract the three most important actors, the director and the keywords associated with that movie.

In [ ]:
# Parse the stringified features into their corresponding python objects
features = ['cast', 'crew', 'keywords', 'genres']
for feature in features:
    metadata[feature] = metadata[feature].apply(literal_eval)

Write functions that will help us to extract the required information from each feature.

In [ ]:
def get_director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return np.nan

In [ ]:
def get_list(x):
    if isinstance(x, list):
        names = [i['name'] for i in x]
        #Check if more than 3 elements exist. If yes, return only first three. If no, return entire list.
        if len(names) > 3:
            names = names[:3]
        return names

    #Return empty list in case of missing/malformed data
    return []

In [ ]:
#metadata_small['crew'][0][0]
#metadata_small['cast'][0][0]
#metadata_small['keywords'][0][0]
#metadata_small['genres'][0][0]

In [ ]:
# Define new director, cast, genres and keywords features that are in a suitable form.
metadata['director'] = metadata['crew'].apply(get_director)

features = ['cast', 'keywords', 'genres']
for feature in features:
    metadata[feature] = metadata[feature].apply(get_list)

In [ ]:
# Print the new features of the first 3 films
metadata[['title', 'cast', 'director', 'keywords', 'genres']].head(3)

,title,cast,director,keywords,genres
0,Toy Story,"[Tom Hanks, Tim Allen, Don Rickles]",John Lasseter,"[jealousy, toy, boy]","[Animation, Comedy, Family]"
1,Jumanji,"[Robin Williams, Jonathan Hyde, Kirsten Dunst]",Joe Johnston,"[board game, disappearance, based on children'...","[Adventure, Fantasy, Family]"
2,Grumpier Old Men,"[Walter Matthau, Jack Lemmon, Ann-Margret]",Howard Deutch,"[fishing, best friend, duringcreditsstinger]","[Romance, Comedy]"


#### Clean Data

In [ ]:
def clean_data(x):
    if isinstance(x, list):
        return [str.lower(i.replace(" ", "")) for i in x]
    else:
        #Check if director exists. If not, return empty string
        if isinstance(x, str):
            return str.lower(x.replace(" ", ""))
        else:
            return ''

In [ ]:
# Apply clean_data function to our features.
features = ['cast', 'keywords', 'director', 'genres']

for feature in features:
    metadata[feature] = metadata[feature].apply(clean_data)

#### Create "metadata soup"

In [ ]:
def create_soup(x):
    return ' '.join(x['keywords']) + ' ' + ' '.join(x['cast']) + ' ' + x['director'] + ' ' + ' '.join(x['genres'])

In [ ]:
# Create a new soup feature
metadata['soup'] = metadata.apply(create_soup, axis=1)

In [ ]:
metadata[['soup']].head(2)

,soup
0,jealousy toy boy tomhanks timallen donrickles ...
1,boardgame disappearance basedonchildren'sbook ...


#### Feature Extraction Using Count Matrix

The same as what we did with our plot description based recommender. One key difference is that we use the CountVectorizer() instead of TF-IDF.

In [ ]:
metadata_small = metadata.head(20000)
count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(metadata_small['soup'])

In [ ]:
count_matrix.shape

(20000, 36471)

#### Calculating Cosine Similarity

In [ ]:
# Compute the Cosine Similarity matrix based on the count_matrix
cosine_sim2 = cosine_similarity(count_matrix, count_matrix)

In [ ]:
# Reset index of our main DataFrame and construct reverse mapping as plot description based recommender
metadata_small = metadata_small.reset_index()
indices = pd.Series(metadata_small.index, index=metadata_small['title'])

#### Reuse our get_recommendations() function to recommend movies based on cosine_sim2

In [ ]:
def get_recommendations(title, cosine_sim=cosine_sim2):
    # Get the index of the movie that matches the title
    idx = indices[title]

    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:11]

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return metadata['title'].iloc[movie_indices]

In [ ]:
print('Movies Similar to "Harry Potter and the Chamber of Secrets" movie:\n\n',get_recommendations('Harry Potter and the Chamber of Secrets'),'\n\n')
print('Movies Similar to "The Matrix" movie:\n\n',get_recommendations('The Matrix'))

Movies Similar to "Harry Potter and the Chamber of Secrets" movie:

 4803         Harry Potter and the Philosopher's Stone
14019          Harry Potter and the Half-Blood Prince
7785         Harry Potter and the Prisoner of Azkaban
10632             Harry Potter and the Goblet of Fire
12019       Harry Potter and the Order of the Phoenix
17593    Harry Potter and the Deathly Hallows: Part 2
16276    Harry Potter and the Deathly Hallows: Part 1
906                                  The Wizard of Oz
5521                                    Spirited Away
8046         The New Adventures of Pippi Longstocking
Name: title, dtype: object 


Movies Similar to "The Matrix" movie:

 6274                    The Matrix Reloaded
6836                 The Matrix Revolutions
6441     Terminator 3: Rise of the Machines
7977                               I, Robot
730                      Ghost in the Shell
1216                         The Terminator
9436                          The Animatrix
13821        

## **Q & A**

### A- What We Have Learned from This Code

We've learned how to improve a movie recommender system using detailed metadata such as actors, directors, genres, and keywords. That's included:

* **Data Integration**: Merging additional metadata (actors, directors, keywords) into the main dataset.
* **Data Preparation**: Parsing and cleaning stringified lists to extract meaningful information.
* **Feature Extraction**: Creating a combined "soup" of metadata for each movie to feed into the recommender.
* **Similarity Computation**: Utilizing CountVectorizer and cosine similarity to measure similarity between movies based on metadata.

### B- The Algorithm Used Description

The algorithm used here is a content-based filtering, works by:
* **Vectorization**: Using CountVectorizer, we convert the combined metadata into a structured format where each movie is represented by its actors, directors, genres, and keywords.
* **Creating a Metadata "Soup"**: All processed metadata is combined into a single representation for each movie, forming what we call a "soup."
* **Matrix Creation**: The CountVectorizer helps us create a matrix where each movie's row represents its metadata. In this matrix, each element indicates the frequency of specific terms (like actors' names, directors, genres, and keywords) within each movie's metadata.
* **Similarity Measurement**: By applying cosine similarity on count matrix, we calculate how similar movies are based on their metadata. This allows us to recommend movies that share common features like actors, directors, genres, and themes.

### C- An Idea to extend this work for collaborative filtering approach

To extend this work with collaborative filtering, we could:
* **Capture User Behavior**: Collect data on how users rate or interact with movies, such as ratings, views, or preferences.
   
* **Build User Profiles**: Create profiles for each user based on their movie interactions and preferences.

* **Collaborative Filtering Algorithms**: Implement neighborhood-based methods within collaborative filtering algorithms.

* **Recommendation Process**: Recommend movies to users based on similarities with others who have similar preferences and behaviors.

### D- Conclusion

In summary, by including detailed information like actors, directors, genres, and keywords in our recommender system, we've made movie recommendations much better. This method gives users more personalized suggestions than traditional systems that only look at plot description.

Additionally, integrating collaborative filtering will allow us to make recommendations that take into account what users like, which can make our suggestions even more accurate.